In [53]:
from text_processing.NGramCounter import NGramCounter
from text_processing.NGramVectorBuilder import NGramVectorBuilder
from data_module.models import Person, Examination, Description, ImageSeries
from data_module.models import Image as Retina_Image
from random import shuffle
import random, re, PIL, csv, cv2, os, time
import numpy as np
from skimage.transform import resize
np.random.seed(7)
import h5py
from keras.utils import to_categorical

In [66]:
class Data_preparer():
    def __init__(self):
        self.load_image_resolution()
    
    def samples_from_file(self, f):
            reader = csv.reader(f.readline().split('\n'), delimiter=',')
            for row in reader:
                train_samples = np.array(row[:-1],dtype='uint8')
                break

            reader = csv.reader(f.readline().split('\n'), delimiter=',')
            for row in reader:
                val_samples = np.array(row[:-1],dtype='uint8')
                break

            reader = csv.reader(f.readline().split('\n'), delimiter=',')
            for row in reader:
                test_samples = np.array(row[:-1],dtype='uint8')
                break
            f.close()
            return train_samples,val_samples,test_samples
    
    def get_ngram_vector_builder(self):
        n = 2; self.outputs = 1000; limit = 0
        ngc = NGramCounter()
        self.n_gram = ngc.get_n_gram_histogram(n,limit,self.outputs)
        return NGramVectorBuilder(self.n_gram)
    
    def load_image_resolution(self):
        width = 1388
        height = 1038
        
        self.img_size_2 = 100
        self.img_size_1 = int(self.img_size_2 * (height / width))

    def get_images_metadata(self,examinations):
            metadata = []
            for exam_id in examinations:
                examination = Examination.objects.get(id=exam_id)
                sequences = ImageSeries.objects.filter(examination_id=exam_id)
                for i in range(len(sequences)):
                    if sequences[i].name.endswith("after_registration") or sequences[i].name.startswith("d") : 
                        continue
                    if sequences[i].name.startswith("left"):
                        side = 'L'
                    else:
                        side = 'R'
                    imgModels = Retina_Image.objects.filter(image_series=sequences[i])
                    imgNum = len(imgModels)
                    if imgNum == 0:
                        continue
                    metadata.append({'examination_id':exam_id,'icd':examination.icd_code,'age': examination.current_age, 'sex':examination.person.sex  ,'side': side, 'series': sequences[i].id, 'first': imgModels[0].id, 'middle': imgModels[int(imgNum/2)].id,'last': imgModels[imgNum-1].id})
                shuffle(metadata)
            return metadata

    def prepare_image(self,_id):
        img = Retina_Image.objects.get(id=_id)
        img = PIL.Image.open(img.image).convert('L')
        arr_img = np.array(img)
        arr_img = self.preprocess_image(arr_img, self.img_size_1, self.img_size_2)
        return arr_img 

    def to_float(self,image):
        func = np.vectorize(lambda x: x / 255.0)
        return func(image)

    def standardization(self,image):
        return (image - np.mean(image)) / np.std(image)

    def preprocess_image(self,image, size_1, size_2, channel=0):
        image = resize(image, (self.img_size_1, self.img_size_2, 1))
        image = self.to_float(image)
        image = self.standardization(image)

        return image

    def create_dataset_and_store(self,hdf5_file, name, metadata, encoded_ages):
        x_name = name+'_x'
        x_shape = (len(metadata), self.img_size_1, self.img_size_2, 3)
        hdf5_file.create_dataset(x_name, x_shape, np.float32)
        
        medic_name = name+'_medic'
        medic_shape = (len(metadata), 93)
        hdf5_file.create_dataset(medic_name, medic_shape, np.float32)
        
        y_name = name+'_y'
        y_shape = (len(metadata),9)
        hdf5_file.create_dataset(y_name, y_shape, np.int8)
        
        meta_name = name+'_metadata'
        meta_shape = (len(metadata),)
        hdf5_file.create_group(meta_name)
        
        for i in range(len(metadata)):
            meta_elem = metadata[i]

            #save X data
            img1 = self.prepare_image(meta_elem['first'])
            img2 = self.prepare_image(meta_elem['middle'])
            img3 = self.prepare_image(meta_elem['last'])

            x = np.dstack((img1,img2,img3))
            hdf5_file[x_name][i, ...] = x[None]
            sex = [1, 0] #M
            if meta_elem['sex'] == 'F':
                sex = [0,1]
            hdf5_file[medic_name][i, ...] = list(self.get_vector_age(encoded_ages,meta_elem['age'])) + sex

            #save Y data
            icds_vector = self.get_icd_encoded(self.onehot_encoded,self.label_encoder,meta_elem['icd'])
            hdf5_file[y_name][i, ...] = icds_vector
            
            #save metadata
            gr = hdf5_file.create_group(meta_name+'/'+str(i))
            for k, v in meta_elem.items():
                gr[k] = v
                

    def store_all_data_in_h5py_file(self):
        
        f = open('splited_data.txt')
        train_samples, val_samples, test_samples = self.samples_from_file(f)

        train_metadata = self.get_images_metadata(train_samples)
        val_metadata = self.get_images_metadata(val_samples)
        test_metadata = self.get_images_metadata(test_samples)

        hdf5_path = './data_icd6.hdf5'
        hdf5_file = h5py.File(hdf5_path, mode='w')
        
        ages = self.get_list_of_ages()
        encoded_ages = to_categorical(ages)
        
        icds = self.get_list_of_icds()
        values = icds
        self.label_encoder = LabelEncoder()
        integer_encoded = label_encoder.fit_transform(values)
        # binary encode
        onehot_encoder = OneHotEncoder(sparse=False)
        integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
        self.onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

        
        self.create_dataset_and_store(hdf5_file,'train_data',train_metadata,encoded_ages)
        self.create_dataset_and_store(hdf5_file,'val_data',val_metadata,encoded_ages)
        self.create_dataset_and_store(hdf5_file,'test_data',test_metadata,encoded_ages)
            
        hdf5_file.close()

    def get_list_of_ages(self):
        ages = set()
        for examin in Examination.objects.filter():
            ages.add(examin.current_age)
        return list(ages)
    
    def get_icd_encoded(self,onehot_encoded,lebel_encoder,icd):
        for el in range(len(onehot_encoded)):
            inverted = label_encoder.inverse_transform([argmax(onehot_encoded[el, :])])
            if inverted == icd:
                return onehot_encoded[el]
            
    def get_list_of_icds(self):
        icds = set()
        for examin in Examination.objects.filter():
            icds.add(examin.icd_code)
        return list(icds)
    
    def get_vector_age(self,encoded, age):
        for el in encoded:
            if argmax(el) == age:
                return el
    
    

In [67]:
data_preparer = Data_preparer()
data_preparer.store_all_data_in_h5py_file()

C:\Users\adam\AppData\Local\conda\conda\envs\inz_retina\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [95]:
from numpy import argmax
def get_list_of_icds():
    icds = set()
    arr = []
    for examin in Examination.objects.filter():
        icds.add(examin.icd_code)
        arr.append(examin.icd_code)
    return list(icds)

icds = get_list_of_icds()
print(icds)
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

values = icds
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

    
def get_icd_encoded(onehot_encoded,lebel_encoder,icd):
    for el in range(len(onehot_encoded)):
        inverted = label_encoder.inverse_transform([argmax(onehot_encoded[el, :])])
        if inverted == icd:
            return onehot_encoded[el]

c = get_icd_encoded(onehot_encoded,label_encoder,'d 31')
print(c)

['h 35.0', 'h 35.3', 'h 57', 'h 31.8', 'h 32', 'h 52.1', 'h 34', 'd 31', 'h 30']
[ 1.  0.  0.  0.  0.  0.  0.  0.  0.]


In [71]:
print(arr)

NameError: name 'arr' is not defined

In [77]:
from keras.utils import to_categorical
# one hot encode
encoded_ages = to_categorical(ages)
def get_vector_age(encoded, age):
    for el in encoded:
        if argmax(el) == age:
            return el
v = list(get_vector_age(encoded_ages,20))
m = v + [0,1]
print(m)
print(len(m))

NameError: name 'ages' is not defined

NameError: name 'roc_curve' is not defined